In [6]:
import pandas as pd
import plotly.express as px  # if needed for further plotting
import re
import numpy as np


def process_resource_room(filepath):
    """
    Processes the resource room Excel file and returns a cleaned and categorized DataFrame.

    Parameters:
        filepath (str): The path to the Excel file (e.g., '/content/CSD - Resource Room.xlsx').

    Returns:
        pd.DataFrame: The processed DataFrame with an added 'Type' column.
    """
    # Read the Excel file
    df = pd.read_excel(filepath)
    Resource_Room = pd.DataFrame(df)
    
    # Remove unwanted columns
    Resource_Room = Resource_Room.drop(labels='Campus', axis=1)
    Resource_Room = Resource_Room.drop(labels='Workshop', axis=1)
    
    # Exclude rows containing 'IMus'
    Resource_Room = Resource_Room[
        ~Resource_Room.apply(lambda row: row.astype(str).str.contains('IMus', case=False, na=False).any(), axis=1)
    ]
    
    # Exclude rows where Lecture, Tutorial, Lab are all 'N'
    Resource_Room = Resource_Room[~(Resource_Room[['Lecture', 'Tutorial', 'Lab']].eq('N').all(axis=1))]
    
    # Fill NA values with 'N' and replace 'N' in Capacity with '20'
    Resource_Room = Resource_Room.fillna('N')
    Resource_Room['Capacity'] = Resource_Room['Capacity'].replace('N', '20')
    
    # Drop rows with Capacity equal to 0
    Resource_Room = Resource_Room.drop(Resource_Room[Resource_Room['Capacity'] == 0].index)
    
    # Categorize as General where Lecture, Tutorial, Lab are all 'Y'
    conditions = [['Y', 'Y', 'Y']]
    columns_to_check = ['Lecture', 'Tutorial', 'Lab']
    General = Resource_Room[
        Resource_Room[columns_to_check].apply(tuple, axis=1).isin(map(tuple, conditions))
    ]
    
    # Exclude specific conditions to form etc_Room (not strictly needed for exclusion logic in this snippet)
    conditions_to_exclude = [['Y', 'Y', 'N'], ['N', 'Y', 'Y']]
    etc_Room = Resource_Room[
        ~Resource_Room[columns_to_check].apply(tuple, axis=1).isin(map(tuple, conditions))
    ]
    
    # Identify Kitchen and Design related rooms
    Kitchen = etc_Room[
        etc_Room.apply(lambda row: row.astype(str).str.contains('Kitchen', regex=False).any(), axis=1)
    ]
    Design = etc_Room[
        etc_Room['Description'].str.contains('DESIGN', case=False, na=False)
    ]
    df_combined = pd.concat([Kitchen, Design])
    
    # Get remaining rows excluding those in df_combined
    LL = etc_Room[
        ~etc_Room.apply(tuple, axis=1).isin(df_combined.apply(tuple, axis=1))
    ]
    
    # Categorize Lab rooms: where [Lecture, Tutorial, Lab] == ['N', 'N', 'Y']
    lab_condition = [['N', 'N', 'Y']]
    Lab = LL[
        LL[columns_to_check].apply(tuple, axis=1).isin(map(tuple, lab_condition)) &
        ~LL.apply(tuple, axis=1).isin(df_combined.apply(tuple, axis=1))
    ]
    
    # Categorize Lecture rooms: where [Lecture, Tutorial, Lab] == ['Y', 'N', 'N']
    lecture_condition = [['Y', 'N', 'N']]
    Lecture = LL[
        LL[columns_to_check].apply(tuple, axis=1).isin(map(tuple, lecture_condition)) &
        ~LL.apply(tuple, axis=1).isin(df_combined.apply(tuple, axis=1))
    ]
    
    # Update df_combined with Lab and Lecture, then get ETC as remaining rows
    df_combined = pd.concat([Lab, Lecture])
    ETC = LL[~LL.apply(tuple, axis=1).isin(df_combined.apply(tuple, axis=1))]
    
    # Assign types
    General['Type'] = 'General'
    Lab['Type'] = 'Lab'
    Kitchen['Type'] = 'Kitchen'
    Design['Type'] = 'Art'
    Lecture['Type'] = 'Lecture'
    ETC['Type'] = 'Etc'
    
    # Combine all categorized DataFrames
    df_final = pd.concat([General, ETC, Kitchen, Design, Lab, Lecture], ignore_index=True)
    df_final = df_final.drop_duplicates(keep='first')
    
    # Override type to 'PBL' if any row contains 'PBL'
    df_final.loc[
        df_final.astype(str).apply(lambda row: row.str.contains('PBL', case=False, na=False).any(), axis=1),
        'Type'
    ] = 'PBL'
    
    return df_final

# Example usage:
filepath = "CSD - Resource Room (5).xlsx"
abc = process_resource_room(filepath)
abc


C:\Users\glasr\AppData\Local\Temp\ipykernel_1500\2485835533.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  General['Type'] = 'General'
C:\Users\glasr\AppData\Local\Temp\ipykernel_1500\2485835533.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Lab['Type'] = 'Lab'
C:\Users\glasr\AppData\Local\Temp\ipykernel_1500\2485835533.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,Resource Code,Description,Capacity,Lecture,Tutorial,Lab,Resource Status,Type
0,MPH - GBS,MPH - [MULTIPURPOSE HALL],100.0,Y,Y,Y,Active,General
1,C404,C404 - [C404],20.0,Y,Y,Y,Active,General
2,C3M04b,C3M04 - [C3M04b-SABE Computer Lab],30.0,Y,Y,Y,Active,General
3,C3M05,C3M05 - [C3M05-Computer Lab],30.0,Y,Y,Y,Active,General
4,C314,C314 - [Microteaching Lab],30.0,Y,Y,Y,Active,General
...,...,...,...,...,...,...,...,...
180,CHEM1,Chemistry 1 Lab,48.0,N,N,Y,Active,Lab
181,CHEM2,Chemistry 2 Lab,48.0,N,N,Y,Active,Lab
182,PHYTO,Phytochemistry Lab,20.0,N,N,Y,Active,Lab
183,Civil Workshop,"Civil Workshop, Level 4 Block E",30.0,N,N,Y,Active,Lab


In [ ]:
import pandas as pd
import plotly.express as px  # if needed for further plotting
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import linear_sum_assignment


def process_resource_room(filepath):
    """
    Processes the resource room Excel file and returns a cleaned and categorized DataFrame.

    Parameters:
        filepath (str): The path to the Excel file (e.g., '/content/CSD - Resource Room.xlsx').

    Returns:
        pd.DataFrame: The processed DataFrame with an added 'Type' column.
    """
    # Read the Excel file
    df = pd.read_excel(filepath)
    Resource_Room = pd.DataFrame(df)
    
    # Remove unwanted columns
    Resource_Room = Resource_Room.drop(labels='Campus', axis=1)
    Resource_Room = Resource_Room.drop(labels='Workshop', axis=1)
    
    # Exclude rows containing 'IMus'
    Resource_Room = Resource_Room[
        ~Resource_Room.apply(lambda row: row.astype(str).str.contains('IMus', case=False, na=False).any(), axis=1)
    ]
    
    # Exclude rows where Lecture, Tutorial, Lab are all 'N'
    Resource_Room = Resource_Room[~(Resource_Room[['Lecture', 'Tutorial', 'Lab']].eq('N').all(axis=1))]
    
    # Fill NA values with 'N' and replace 'N' in Capacity with '20'
    Resource_Room = Resource_Room.fillna('N')
    Resource_Room['Capacity'] = Resource_Room['Capacity'].replace('N', '20')
    
    # Drop rows with Capacity equal to 0
    Resource_Room = Resource_Room.drop(Resource_Room[Resource_Room['Capacity'] == 0].index)
    
    # Categorize as General where Lecture, Tutorial, Lab are all 'Y'
    conditions = [['Y', 'Y', 'Y']]
    columns_to_check = ['Lecture', 'Tutorial', 'Lab']
    General = Resource_Room[
        Resource_Room[columns_to_check].apply(tuple, axis=1).isin(map(tuple, conditions))
    ]
    
    # Exclude specific conditions to form etc_Room (not strictly needed for exclusion logic in this snippet)
    conditions_to_exclude = [['Y', 'Y', 'N'], ['N', 'Y', 'Y']]
    etc_Room = Resource_Room[
        ~Resource_Room[columns_to_check].apply(tuple, axis=1).isin(map(tuple, conditions))
    ]
    
    # Identify Kitchen and Design related rooms
    Kitchen = etc_Room[
        etc_Room.apply(lambda row: row.astype(str).str.contains('Kitchen', regex=False).any(), axis=1)
    ]
    Design = etc_Room[
        etc_Room['Description'].str.contains('DESIGN', case=False, na=False)
    ]
    df_combined = pd.concat([Kitchen, Design])
    
    # Get remaining rows excluding those in df_combined
    LL = etc_Room[
        ~etc_Room.apply(tuple, axis=1).isin(df_combined.apply(tuple, axis=1))
    ]
    
    # Categorize Lab rooms: where [Lecture, Tutorial, Lab] == ['N', 'N', 'Y']
    lab_condition = [['N', 'N', 'Y']]
    Lab = LL[
        LL[columns_to_check].apply(tuple, axis=1).isin(map(tuple, lab_condition)) &
        ~LL.apply(tuple, axis=1).isin(df_combined.apply(tuple, axis=1))
    ]
    
    # Categorize Lecture rooms: where [Lecture, Tutorial, Lab] == ['Y', 'N', 'N']
    lecture_condition = [['Y', 'N', 'N']]
    Lecture = LL[
        LL[columns_to_check].apply(tuple, axis=1).isin(map(tuple, lecture_condition)) &
        ~LL.apply(tuple, axis=1).isin(df_combined.apply(tuple, axis=1))
    ]
    
    # Update df_combined with Lab and Lecture, then get ETC as remaining rows
    df_combined = pd.concat([Lab, Lecture])
    ETC = LL[~LL.apply(tuple, axis=1).isin(df_combined.apply(tuple, axis=1))]
    
    # Assign types
    General['Type'] = 'General'
    Lab['Type'] = 'Lab'
    Kitchen['Type'] = 'Kitchen'
    Design['Type'] = 'Art'
    Lecture['Type'] = 'Lecture'
    ETC['Type'] = 'Etc'
    
    # Combine all categorized DataFrames
    df_final = pd.concat([General, ETC, Kitchen, Design, Lab, Lecture], ignore_index=True)
    df_final = df_final.drop_duplicates(keep='first')
    
    # Override type to 'PBL' if any row contains 'PBL'
    df_final.loc[
        df_final.astype(str).apply(lambda row: row.str.contains('PBL', case=False, na=False).any(), axis=1),
        'Type'
    ] = 'PBL'
    
    return df_final


def preprocess_offer(file_path):
    # 엑셀 파일을 DataFrame으로 불러옵니다.
    offer = pd.read_excel(file_path)

    offer.drop(columns=['Unnamed: 0'], inplace=True)
    offer.drop(offer.index[1157], axis=0, inplace=True)
    offer['Capacity'] = offer['Capacity'].astype(int)
    offer['Min Per Session'] = offer['Min Per Session'].astype(int)

    offer['Session'] = offer['Session'].str.upper()
    offer['Lecturer'] = offer['Lecturer'].str.upper()
    offer['CourseCode'] = offer['CourseCode'].str.upper()
    offer['FacultyCode'] = offer['FacultyCode'].str.upper()

    offer['Session'] = offer['Session'].str.replace(r'\(S\)', '', regex=True)

    # "COMBINED TO /"를 복사하여 error 테이블에 저장. 이후 원본 데이터에서 삭제.
    mask = offer['Session'].str.contains("COMBINED TO /", na=False)

    # error 데이터프레임에 해당 행들을 복사하여 저장합니다.
    error = offer[mask].copy()

    # 원본 offer 데이터프레임에서 해당 행들을 삭제합니다.
    offer.drop(offer[mask].index, inplace=True)

    #"GROUP A", "GROUP B", "GROUP C" 등 해당하는 행을 찾고, Session 값을 "LECTURE"로 변경
    offer.loc[offer['Session'].str.contains(r'^GROUP\s+[A-Z]$', na=False), 'Session'] = 'LECTURE'

    group_cols = ['CourseCode', 'FacultyCode', 'Session', 'Capacity', 'Min Per Session', 'Lecturer']

    # 그룹별로 중복된 행을 처리합니다.
    for _, group in offer.groupby(group_cols):
        if len(group) > 1:  # 중복 그룹인 경우
            # 원래 Session 값을 앞뒤 공백 제거 후 사용합니다.
            orig_session = group.iloc[0]['Session'].strip()
            # Session 값이 숫자로 끝나는지 검사 (예: "TUTORIAL 1")
            m = re.search(r'^(.*?)(\d+)$', orig_session)
            if m:
                base = m.group(1).strip()  # 문자 부분 (예: "TUTORIAL")
                start_num = int(m.group(2))  # 기존에 붙은 숫자 (예: 1)
            else:
                base = orig_session
                start_num = 1

            # 그룹 내 각 행에 대해 순차적으로 번호를 붙여 Session 값을 변경합니다.
            for offset, idx in enumerate(group.index):
                new_session = f"{base} {start_num + offset}"
                offer.at[idx, 'Session'] = new_session

    # (초기 전제: offer DataFrame과 error DataFrame이 존재함; error가 없으면 빈 DataFrame 생성)
    if 'error' not in globals():
        error = pd.DataFrame()

    ### Step 1. offer에서 "COMBINED TO" 행 추출 → child_class로 저장, offer에서는 제거
    child_class = offer[offer['Session'].str.contains('COMBINED TO', na=False)].copy()
    offer.drop(child_class.index, inplace=True)

    ### Step 2. child_class의 Session 열에서 과목코드 추출
    child_class['extracted_code'] = child_class['Session'].str.extract(r'COMBINED TO\s+([A-Z0-9]+)', expand=False)

    ### Step 3. child_class 내에서 오류 및 정상/후손 분리

    # (A) "자기 자신 참조" 조건: 추출한 코드가 해당 행의 CourseCode와 같으면 → 오류
    self_ref = child_class['extracted_code'] == child_class['CourseCode']

    # (B) 정상(child) 조건: 자기 자신 참조가 아니고, 추출한 코드가 offer의 CourseCode에 존재하는 경우
    valid_child_mask = (~self_ref) & (child_class['extracted_code'].isin(offer['CourseCode']))

    # (C) 후손(grandchild) 후보: 자기 자신 참조가 아니고, 추출한 코드가 offer에는 없지만, child_class의 CourseCode에는 존재하는 경우  
    #     (즉, 부모(대상 CourseCode)가 offer에 없으므로 자식끼리 연결되어 있음을 의미)
    valid_grand_mask = (~self_ref) & (~child_class['extracted_code'].isin(offer['CourseCode'])) & \
                        (child_class['extracted_code'].isin(child_class['CourseCode']))

    # (D) 오류 조건:  
    #     - 자기 자신 참조인 경우  
    #     - 또는 추출한 코드가 offer에도 child_class에도 존재하지 않는 경우
    error_mask = self_ref | ((~child_class['extracted_code'].isin(offer['CourseCode'])) & 
                               (~child_class['extracted_code'].isin(child_class['CourseCode'])))

    # child_class 오류 처리: 추출
    child_error = child_class[error_mask].copy()

    # 정상 child_class (valid_child) 추출
    valid_child = child_class[valid_child_mask].copy()

    # 후보 후손(grandchild) 추출
    candidate_grand = child_class[valid_grand_mask].copy()

    ### Step 4. grand_child_class 내에서 추가 오류 처리
    # 오류 조건 (grandchild): 자기 자신 참조 → 오류
    grand_self_ref = candidate_grand['extracted_code'] == candidate_grand['CourseCode']
    error_grand = candidate_grand[grand_self_ref].copy()

    # 최종 grand_child_class: 후보에서 자기 자신 참조 오류 제거
    grand_child_class = candidate_grand[~grand_self_ref].copy()

    # 최종 error: 기존 child 오류와 후손에서 발생한 오류를 모두 누적
    error_df = pd.concat([child_error, error_grand], ignore_index=True)
    error = pd.concat([error, error_df], ignore_index=True)

    ### 최종 정상 데이터
    # child_class는 valid_child로 업데이트 (offer의 부모가 존재하는 경우)
    child_class = valid_child.copy()
    # grand_child_class는 위에서 구한 대로

    # error DataFrame이 없으면 빈 DataFrame 생성
    if 'error' not in globals():
        error = pd.DataFrame()

    ##############################################
    # 교차검증 1: child_class에서 자기 자신 참조하는 행 처리
    ##############################################
    child_self_ref_mask = (child_class['extracted_code'] == child_class['CourseCode'])
    child_self_ref_errors = child_class[child_self_ref_mask].copy()
    error = pd.concat([error, child_self_ref_errors], ignore_index=True)
    child_class = child_class[~child_self_ref_mask].copy()

    ##############################################
    # 교차검증 2: child_class에서 추출한 과목코드가 offer의 CourseCode에 없음(부모 없음)
    ##############################################
    child_missing_parent_mask = ~child_class['extracted_code'].isin(offer['CourseCode'])
    child_missing_parent_errors = child_class[child_missing_parent_mask].copy()
    error = pd.concat([error, child_missing_parent_errors], ignore_index=True)
    child_class = child_class[~child_missing_parent_mask].copy()

    ##############################################
    # 교차검증 3: offer에서 자기 자신 참조하는 행 처리
    # (offer에 COMBINED TO가 남아있다면 해당 행에 대해 extracted_code를 추출)
    ##############################################
    offer_combined_mask = offer['Session'].str.contains('COMBINED TO', na=False)
    if offer_combined_mask.any():
        offer.loc[offer_combined_mask, 'extracted_code'] = offer.loc[offer_combined_mask, 'Session'].str.extract(r'COMBINED TO\s+([A-Z0-9]+)', expand=False)
        offer_self_ref_mask = (offer['extracted_code'] == offer['CourseCode'])
        offer_self_ref_errors = offer[offer_self_ref_mask].copy()
        error = pd.concat([error, offer_self_ref_errors], ignore_index=True)
        offer.drop(offer[offer_self_ref_mask].index, inplace=True)

    ##############################################
    # 교차검증 4: grand_child_class에서 자기 자신 참조하는 행 처리
    ##############################################
    grand_self_ref_mask = (grand_child_class['extracted_code'] == grand_child_class['CourseCode'])
    grand_self_ref_errors = grand_child_class[grand_self_ref_mask].copy()
    error = pd.concat([error, grand_self_ref_errors], ignore_index=True)
    grand_child_class = grand_child_class[~grand_self_ref_mask].copy()

    ##############################################
    # 교차검증 5: grand_child_class에서 추출한 과목코드가 offer의 CourseCode에 없음(부모 없음)
    ##############################################
    grand_missing_parent_mask = ~grand_child_class['extracted_code'].isin(offer['CourseCode'])
    grand_missing_parent_errors = grand_child_class[grand_missing_parent_mask].copy()
    error = pd.concat([error, grand_missing_parent_errors], ignore_index=True)
    grand_child_class = grand_child_class[~grand_missing_parent_mask].copy()

    ##############################################
    # 교차검증 6: 부모(offer 또는 child_class)에서 드랍된 행과 연결된 자식(또는 손자) 행 처리
    # 드랍된 행들의 CourseCode를 모아서, 이 코드를 extracted_code로 사용하는 행이 있다면 error로 처리
    ##############################################
    # 지금까지 error에 추가된 행들의 CourseCode를 모읍니다.
    dropped_codes = set(error['CourseCode'].unique())

    # child_class에서 연결된 자식 행 검사
    child_linked_mask = child_class['extracted_code'].isin(dropped_codes)
    child_linked_errors = child_class[child_linked_mask].copy()
    error = pd.concat([error, child_linked_errors], ignore_index=True)
    child_class = child_class[~child_linked_mask].copy()

    # grand_child_class에서 연결된 손자 행 검사
    grand_linked_mask = grand_child_class['extracted_code'].isin(dropped_codes)
    grand_linked_errors = grand_child_class[grand_linked_mask].copy()
    error = pd.concat([error, grand_linked_errors], ignore_index=True)
    grand_child_class = grand_child_class[~grand_linked_mask].copy()

    # child_class의 각 행을 순회하며 조건에 맞는 offer 행의 Capacity에 child_class의 Capacity를 합산합니다.
    for idx, child_row in child_class.iterrows():
        # 조건: offer의 CourseCode, Min Per Session, Lecturer가 child_row의 extracted_code, Min Per Session, Lecturer와 일치
        mask = (
            (offer['CourseCode'] == child_row['extracted_code']) &
            (offer['Min Per Session'] == child_row['Min Per Session']) &
            (offer['Lecturer'] == child_row['Lecturer'])
        )
        # 일치하는 행이 있다면, offer의 Capacity에 child_row의 Capacity를 더함
        if mask.any():
            offer.loc[mask, 'Capacity'] += child_row['Capacity']

    # 기존 조건
    conditions = [
        offer['Session'].str.contains('LECTURE & TUTORIAL', na=False, case=False),
        offer['Session'].str.contains('TUTORIAL', na=False, case=False),
        offer['Session'].str.contains('LECTURE', na=False, case=False),
        offer['Session'].str.contains('LAB', na=False, case=False),
        offer['Session'].str.contains('PBL 1', na=False, case=False),
        offer['Session'].str.contains('HAND DRAWING|CAD DRAWING 2|CAD DRAWING 1', na=False, case=False),
        offer['Session'].str.contains('KITCHEN', na=False, case=False),
        offer['Session'].str.contains('OPTOM CLINIC|SOO - EXAM CLINIC', na=False, case=False)
    ]

    # 기존 선택지 + 새 선택지
    choices = [
        'GENERAL',
        'TUTORIAL',
        'LECTURE',
        'LAB',
        'PBL',
        'ART',
        'KITCHEN',
        'LECTURE'
    ]

    # 조건을 적용하여 'Category' 열 생성
    offer['Category'] = np.select(conditions, choices, default=np.nan)
    
    return offer


def expand_room_slots(df_final):
    time_slots = ['8-11', '13-16', '18-21']
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']

    expanded_rooms = []
    for _, row in df_final.iterrows():
        for day in days:
            for time in time_slots:
                expanded_rooms.append({
                    'Resource Code': row['Resource Code'],
                    'Type': row['Type'],
                    'Capacity': row['Capacity'],
                    'Assigned Day': day,
                    'Assigned Time Slot': time,
                    'Lecture': row.get('Lecture', 'N'),
                    'Tutorial': row.get('Tutorial', 'N'),
                    'Lab': row.get('Lab', 'N')
                })

    return pd.DataFrame(expanded_rooms)

def preprocess_offers(offer, max_room_capacity):
    expanded_offers = []
    for _, row in offer.iterrows():
        remaining_students = row['Capacity']
        total_duration = row['Min Per Session']
        session_count = 1

        while remaining_students > 0:
            assigned_capacity = min(max_room_capacity, remaining_students)
            remaining_students -= assigned_capacity

            remaining_duration = total_duration
            sub_session = 1

            while remaining_duration > 0:
                session_duration = min(180, remaining_duration)
                remaining_duration -= session_duration

                expanded_offers.append({
                    'Course Code': f"{row['CourseCode']}_S{session_count}_D{sub_session}",
                    'Capacity': assigned_capacity,
                    'Min Per Session': session_duration,
                    'Category': row['Category'],
                    'Lecturer': row['Lecturer']
                })
                sub_session += 1

            session_count += 1

    return pd.DataFrame(expanded_offers)

def match_classroom_type(df_final, offer):
    offer['Category'] = offer['Category'].str.lower()
    df_final['Type'] = df_final['Type'].str.lower()
    df_final['Type'] = df_final.apply(lambda x: 'tutorial' if x['Type'] == 'etc' else x['Type'], axis=1)
    return df_final, offer

def create_cost_matrix(df_final, offer):
    num_offers = len(offer)
    num_rooms = len(df_final)
    max_size = max(num_offers, num_rooms)
    cost_matrix = np.full((max_size, max_size), 1e6)

    df_final['Capacity'] = df_final['Capacity'].astype(int)
    offer['Capacity'] = offer['Capacity'].astype(int)

    # 🔹 교수님이 특정 시간에 중복되지 않도록 하기 위한 매핑
    professor_time_map = {}

    for i, offer_row in offer.iterrows():
        for j, room_row in df_final.iterrows():
            assigned_time = (room_row['Assigned Day'], room_row['Assigned Time Slot'])
            professor = offer_row['Lecturer']

            if (offer_row['Category'] == room_row['Type'] or
                (offer_row['Category'] in ['lecture', 'tutorial'] and room_row['Type'] in ['general', 'lecture']) or
                (offer_row['Category'] == 'lecture' and room_row['Type'] == 'tutorial' and room_row['Lecture'] == 'Y')):

                if room_row['Capacity'] >= offer_row['Capacity']:
                    # 🔹 같은 시간대에 이미 배정된 교수님이 있는 경우 패널티를 줌
                    if assigned_time in professor_time_map and professor in professor_time_map[assigned_time]:
                        cost_matrix[i, j] = 1e6  # 불가능한 배정 (큰 값)
                    else:
                        cost_matrix[i, j] = room_row['Capacity'] - offer_row['Capacity']

    return cost_matrix


def assign_classes(df_final, offer):
    df_final_expanded = expand_room_slots(df_final)
    cost_matrix = create_cost_matrix(df_final_expanded, offer)

    num_offers, num_rooms = cost_matrix.shape
    max_size = max(num_offers, num_rooms)

    if num_offers < max_size:
        cost_matrix = np.vstack([cost_matrix, np.full((max_size - num_offers, max_size), 1e6)])
    if num_rooms < max_size:
        cost_matrix = np.hstack([cost_matrix, np.full((max_size, max_size - num_rooms), 1e6)])

    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    assignments = []
    professor_time_map = {}  # 🔹 교수님 시간 중복 체크

    for r, c in zip(row_ind, col_ind):
        if r < len(offer) and c < len(df_final_expanded) and cost_matrix[r, c] != 1e6:
            assigned_day = df_final_expanded.iloc[c]['Assigned Day']
            assigned_time = df_final_expanded.iloc[c]['Assigned Time Slot']
            professor = offer.iloc[r]['Lecturer']

            # 🔹 같은 시간에 같은 교수님이 이미 배정된 경우 무조건 제외
            if (assigned_day, assigned_time) in professor_time_map and professor in professor_time_map[(assigned_day, assigned_time)]:
                continue  # 🚨 이 배정은 무효! 스킵!

            # 🔹 교수님 배정 확정 (시간대 저장)
            if (assigned_day, assigned_time) not in professor_time_map:
                professor_time_map[(assigned_day, assigned_time)] = set()
            professor_time_map[(assigned_day, assigned_time)].add(professor)

            assignments.append({
                'Group': offer.iloc[r]['Course Code'],
                'Category': offer.iloc[r]['Category'],
                'Lecturer': professor,
                'Assigned Room': df_final_expanded.iloc[c]['Resource Code'],
                'Room Type': df_final_expanded.iloc[c]['Type'],
                'Group Capacity': offer.iloc[r]['Capacity'],
                'Room Capacity': df_final_expanded.iloc[c]['Capacity'],
                'Assigned Day': assigned_day,
                'Assigned Time Slot': assigned_time,
                'Lecture': df_final_expanded.iloc[c]['Lecture'],
                'Tutorial': df_final_expanded.iloc[c]['Tutorial'],
                'Lab': df_final_expanded.iloc[c]['Lab'],
                'Duration (min)': offer.iloc[r]['Min Per Session']
            })

    return pd.DataFrame(assignments)


def generate_assigned_schedule(df_final, offer):
    """
    Generates the assigned schedule by processing offers and matching them with available rooms.

    Parameters:
        df_final (pd.DataFrame): The processed resource room DataFrame.
        offer (pd.DataFrame): The course offer DataFrame.

    Returns:
        pd.DataFrame: A DataFrame containing the final class assignments.
    """
    # Determine the maximum room capacity from the resource room DataFrame
    max_room_capacity = df_final['Capacity'].astype(int).max()
    
    # Process the offers by expanding them based on room capacity
    offer = preprocess_offers(offer, max_room_capacity)
    
    # Align the room types and offer categories for matching
    df_final, offer = match_classroom_type(df_final, offer)
    
    # Assign classes based on the constructed cost matrix and constraints
    assigned_schedule = assign_classes(df_final, offer)
    
    return assigned_schedule

# Example usage:
# assigned_schedule = generate_assigned_schedule(df_final, offer)
# print(assigned_schedule)
